# PetitJean Index of Chemical Molecules
> **PetitjeanIndex:** a graph-theoretical shape coefficient

> **Reference:** 
Applications of the radius-diameter diagram to the classification of topological and geometrical shapes of chemical compounds
Michel Petitjean
Journal of Chemical Information and Computer Sciences 1992 32 (4), 331-337
DOI: 10.1021/ci00008a012


**Done By:** 

**Elias T. Abboud**   

---

## Goals

#### Chemical structures have two shape coefficients. The first is based upon graph theory and is calculated from the compound’s atom-bond graph. The other is the geometrical shape coefficient, which is determined by its geometrical shape - which we wish to measure.

#### This workflow allows you to obtain the petitjean index of a chemical molecule in a MOL file, or a combination of those in an SDF file, or a group of mixed MOL and SDF files.

#### In addition, you can obtain 2D images of the molecules in your MOL & SDF files - saved in an output images.zip
---

## I - Loading Libraries

In [117]:
## Install & Load Required Libraries
## NetworkX is a Python package for the creation,
## manipulation, and study of the structure,
## dynamics, and functions of complex networks.
## To install: pip install networkx
import networkx as nx

## Matplot is a Python package
## for plotting purposes.
## Installed by Default
import matplotlib.pyplot as plt

## Glob will be used to load
## all files in a directory.
## Installed by Default
import glob


## OS - File Size
import os

## Numpy for arrays manipulation
import numpy as np

## re for Regex and searching
import re

## csv for output file
import csv

## II - Loading Required Functions

### Reading MOL Data into Graph

In [118]:
def readMolIntoGraph(file_name):
    """
        Function: Import a .mol file into a Graph
        
            Args:
                file_name (str): path of .mol file

            Returns:
                analyzeMolData output
                returns a (1) graph, (2) positions of atoms,
                        a (3) mol id - matches with chembl
    """
    mol_data = []
    myfile = open(file_name, "r")
    while myfile:
        line  = myfile.readline()
        mol_data.append(line.strip())
        if line == "":
            break
    myfile.close() 
    mol_data
    return(analyzeMolData(mol_data))

### Reading SDF Data into Graph

In [119]:
def readSDFIntoGraph(file_name):
    """
        Function: Import a .sdf file into a list of Graphs
        
            Args:
                file_name (str): path of .sdf file

            Returns:
                list of analyzeMolData outputs
                returns a list of
                        (1) graph, 
                        (2) positions of atoms,
                        (3) mol id - matches with chembl
    """
    ## Open file & Initialize empty lists
    myfile = open(file_name, "r")
    output_graphs = []
    output_positions = []
    output_ids = []
    mol_data = []
    
    ## Get file size of sdf, this is to warn
    ## This is to warn the user in case file
    ## is extremely large (>0.5GB)
    file_size = os.path.getsize(file_name)
    if file_size > 500000:
       print("Detected an large sdf file... size is " + str(file_size//1000) + "MB")
       num = int(input("Please select how many molecules want to preview (-1 for all): "))
    else:
       num = -1
    
    ## If num was set to -1
    ## then preview all sdf
    if num == -1:
        num = 10000
    
    ## Set a counter - to count how many
    ## molecules have been previewed so far
    counter = 0
    while counter < num and myfile:
        # Boolean to check if new line
        # belongs same molecule.
        # This helps solve the problem
        # of multiple molecules in an sdf
        not_new = True
        while myfile and not_new:
         line  = myfile.readline()
         
         # Add line into mol_data
         mol_data.append(line.strip())

         # When $$$$ is encountered it means a
         # new molecule is encountered then:
         # ---> stop reading into mol_data
         #      & build the graph & pos.
         if "$$$$" in line:
             ## call analyzeMolData(mol_data)
             G, positions, mol_id = analyzeMolData(mol_data)
             ## add new outputs to outputs lists
             output_graphs.append(G)
             output_positions.append(positions)
             output_ids.append(mol_id)
             not_new = False
             ## reset .mol data to find new molecules
             mol_data = []
             continue
         if line == "":
            break
        mol_data
        counter += 1
    myfile.close() 
    return(output_graphs, output_positions, output_ids)

---
### Analyze Mol Data

> After Reading the Data from a .mol file and .sdf file, we **(a) store the atom bonding the into a Graph**, taking into consideration the **(b) removal of Hydrogen atoms and Hydrogen bonds**, and **(c) storing the positions of atoms for plotting purposes.**

In [120]:
def analyzeMolData(mol_data):
    """
        Function: Store mol bomding data into a graph
                    and get positions of atoms.
        
            Args:
                mol_data (list): list containing lines
                                 of files .mol & .sdf
                                 by readSDFIntoGraph()
                                 or by readMolIntoGraph()

            Returns:
                G (Graph of networkx library): graph representing
                                    atom-bonding in the molecule.
                positions (dictionary): holds data {atom: 2D position}
                final_id (str): represents the ID of the molecule.
    """
    ## Get Positions - Regex
    ## XYZ Block Filter
    xyz_block_filter = re.compile("-?(?:\d+.\d+)\s+-?(?:\d+.\d+)\s+-?(?:\d+.\d+)\s+")   
    
    ## XYZ Block Extraction
    xyz_block = list(filter(xyz_block_filter.search, mol_data))
    xyz_block
    ## ---------------------------

    ## Get ID of molecule - Regex
    final_id = 0
    ## Check if ID is a CHEMBL ID
    chembl_detect = re.compile("CHEMBL")
    chembl_id = list(filter(chembl_detect.search,mol_data))
    chembl_id
    
    # Other ID Format/Also can be searched on ChemBL
    other_id_detect = re.compile("^[A-Z]\d+-\d+")
    other_id = list(filter(other_id_detect.search,mol_data))
    other_id
    
    ## Since Chembl has different binding section format
    if len(chembl_id)!=0:
       bond_block_filter = re.compile("^\d+\s+\d+\s+\d+\s+0$")
       final_id = chembl_id
    else:
       bond_block_filter = re.compile("^\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+$")
       final_id = other_id
    ## ---------------------------
    
    ## Get Atom Bonding - Regex
    bond_block = list(filter(bond_block_filter.search, mol_data))      
    bond_block
    ## ---------------------------
    
    ## Extract Atoms & Positions
    ## Initialize Atoms List
    ## Initialize Positions Dictionary
    atoms = []
    positions = {}
    
    ## Populate Data Structures
    atom_count = 1
    for entry in xyz_block:
        ## Get & Add Atoms
        atom_filter = re.compile("[a-zA-Z]+")
        atom = re.search(atom_filter,entry)
        atom = atom.group()
        atoms.append(atom)
        ## Get & Add Positions
        ## Just X and Y, for 2D representation
        entry = entry.split()
        positions[atom_count] = np.array([float(entry[0]),float(entry[1])])
        atom_count = atom_count + 1
    ## ---------------------------   
        
    ## Build the Graph
    G = nx.Graph()
    bonds = []
    positions_filtered = {}
    for entry in bond_block:
        ## Split entry (line) by space
        entry = entry.split()
        ## Get first and second atoms
        ## these atoms have a bond
        atom1 = int(entry[0])
        atom2 = int(entry[1])
        ## Skip Hydrogen Bonds
        if atoms[atom1-1] == "H" or atoms[atom2-1] == "H":
            continue
        ## Add Atoms that aren't Hydrogen to the Graph
        else:
            G.add_node(atom1, name=atoms[atom1-1])
            G.add_node(atom2, name=atoms[atom2-1])
            ## Add Edge, weight is set to #bonds
            ## The weight is x2 to show more on
            ## the graph/figure produced.
            G.add_edge(atom1, atom2, weight = int(entry[2])*2)
            
            ## Get Positions of atoms - exclude Hydrogen
            positions_filtered[atom1] = positions[atom1]
            positions_filtered[atom2] = positions[atom2]
    
    ## Return the Graph, 
    ## Positions Dictionary
    ## & ID of Molecule
    return(G, positions_filtered, final_id)

---
### Get PetitJeanIndex

> First, we calculate the Eccentricity of every node in Graph G:

$For\ every\ node\ x \in G: Eccentricity(x) = max(d(x,y))\ for\ all\ y\ in\ Nodes\ of\ G\$

> Then we calculate the **Radius** and the **Diameter** of the Graph

$radius\ =\ min(Eccentricity(x_i)\ |\ i\ \in [1,N])$
$diameter\ =\ max(Eccentricity(x_i)\ |\ i\ \in [1,N])$

> Finally, we calculate the PetitJeanIndex

$PJI\ =\frac{diameter - radius}{radius}$


In [121]:
def getPJI(G):
    """
        Function: calculates PJI from Graph
        
            Args:
                G (Graph of networkx library): represents atom-binding

            Returns:
                PJI = Diameter(G)-Radius(G)/Radius(Graph)
    """
    ecc = nx.eccentricity(G).values()
    radius = min(ecc)
    diameter = max(ecc)
    print("Molecule has R="+str(radius)+" and D="+str(diameter))
    numerator = diameter - radius
    dinom = radius
    pji = numerator/dinom
    print("PJI = " + str(pji))
    return(pji)   

---
### Draw Molecule in 2D

In [122]:
def drawGraph(G, positions_filtered):
    """
        Function: Draws the Graph in 2D
        
            Args:
                G (Graph of networkx library): represents atom-binding
                positions_filtered(dictionary): {atom: 2D position}

            Returns: Void
    """
    edges = G.edges()    
    weights = [G[u][v]['weight'] for u,v in edges]
    names = nx.get_node_attributes(G, 'name')
    import matplotlib.pyplot as plt
    nx.draw(G, positions_filtered, labels=names,width=weights)

---
### Read Molecules from Directory & Get PJI


In [123]:
def readMolecules(dir_path, get_figures):
    """
        Function: 1- Reads .mol or .sdf Files & Calculates PJI
                  2- Gets 2D drawings of the
                     molecules if get_figures = True
        
            Args:
                dir_path (str): path of data (.mol and .sdf files)
                get_figures (boolean): if True, outputs a zip of 2D figures.

            Returns: Void
    """
    dictionary_pji = {}
    ## Get all files in directory
    for file_name in glob.iglob(dir_path+"/*"):
        ## Initialize Empty Graph
        graphs = []
        
        ## Check if File is .sdf
        ## Read an SDF - Get PJI
        if file_name.endswith(".sdf"):
            # If SDF, get lists of Graphs and Positions and IDs
            list_of_graphs, list_of_positions, list_of_ids = readSDFIntoGraph(file_name)
           
            # For each molecule:
            for i in range(0, len(list_of_graphs)):
                # Get G, pos and IDs
                G = list_of_graphs[i]
                pos = list_of_positions[i]
                mol_id = list_of_ids[i]
                
                # Get PJI
                print("\n==========\nMolecule ID: " + str(mol_id[0]))
                pji = getPJI(G)
                
                # Save Image with ID & PJI
                if get_figures:
                    from os import path
                    if not path.exists("images"):
                        os.mkdir("images")
                    drawGraph(G, pos)
                    plt.savefig("images\\"+file_name.split(".")[0].split("\\")[-1]+"ID"+str(mol_id)+"_PJI"+str(pji)+".png")
                    plt.clf()
                dictionary_pji[mol_id[0]] = pji
                
        ## Check if file is .mol
        elif file_name.endswith(".mol"):
            ## Get G, positions and IDs
            G,positions_filtered, mol_id = readMolIntoGraph(file_name)
            
            print("\n===========\nMolecule ID: " + str(mol_id[0]))
            pji = getPJI(G)
            if get_figures:
                from os import path
                if not path.exists("images"):
                    os.mkdir("images")
                drawGraph(G, positions_filtered)
                plt.savefig("images\\"+file_name.split(".")[0].split("\\")[-1]+"ID"+str(mol_id)+"_PJI"+str(pji)+".png")
                plt.clf()
            dictionary_pji[mol_id[0]] = pji
        else:
            continue
            
        field_names = ["mol_id", "pji"]
        with open('results_pji.csv', 'w') as f:
            for key in dictionary_pji.keys():
                f.write("%s, %s\n" % (key, dictionary_pji[key]))

---
---
---

# II- Testing

>> The test is ran on 4 .mol files and 1 sdf files that contains multiple molecules.

In [124]:
## Run - Do not Save Images
## Please Specify your path here
path = "data"
get_images = False
readMolecules(path, get_images)


Molecule ID: F1903-0751
Molecule has R=4 and D=8
PJI = 1.0

Molecule ID: F1903-9899
Molecule has R=4 and D=8
PJI = 1.0

Molecule ID: F1903-9900
Molecule has R=4 and D=8
PJI = 1.0

Molecule ID: F1904-0726
Molecule has R=2 and D=4
PJI = 1.0

Molecule ID: F1903-9899
Molecule has R=4 and D=8
PJI = 1.0

Molecule ID: F1903-9900
Molecule has R=4 and D=8
PJI = 1.0

Molecule ID: F1904-0726
Molecule has R=5 and D=10
PJI = 1.0

Molecule ID: F1904-0909
Molecule has R=4 and D=8
PJI = 1.0

Molecule ID: F1904-9831
Molecule has R=5 and D=10
PJI = 1.0

Molecule ID: F1905-3672
Molecule has R=5 and D=9
PJI = 0.8

Molecule ID: F1907-9253
Molecule has R=6 and D=12
PJI = 1.0

Molecule ID: F1911-0295
Molecule has R=5 and D=9
PJI = 0.8

Molecule ID: F1911-2933
Molecule has R=5 and D=10
PJI = 1.0

Molecule ID: F1911-3029
Molecule has R=5 and D=9
PJI = 0.8

Molecule ID: F1903-0754
Molecule has R=3 and D=5
PJI = 0.6666666666666666

Molecule ID: CHEMBL4594434
Molecule has R=7 and D=14
PJI = 1.0

Molecule ID: CHE

<Figure size 432x288 with 0 Axes>